# 🚀 Solana Meme-Coin Sniper Bot\n\nBot automat pentru trading de meme-coins pe blockchain-ul Solana.\n\n## ⚠️ AVERTISMENT\n- Trading-ul de crypto implică **RISC FOARTE MARE**\n- Poți pierde **TOȚI** banii investiți\n- Folosește doar bani pe care îți permiți să îi pierzi\n- Începe cu sume mici pentru teste (<$50)

## 📦 Pasul 1: Instalare și Setup

In [ ]:
# Clonăm repository-ul
!git clone https://github.com/YOUR_USERNAME/solana-sniper-bot.git
%cd solana-sniper-bot

# Instalăm dependințele
!pip install -r requirements.txt -q
!pip install base58 -q

print("✅ Instalare completă!")

## 🔑 Pasul 2: Configurare Wallet și API Keys\n\nAi nevoie de:\n1. **Moralis API Key** - [Obține gratuit aici](https://moralis.io/)\n2. **Phantom Wallet Private Key** - Exportă din Settings → Security\n3. **Helius API Key** (opțional) - [Obține aici](https://helius.dev/)

In [ ]:
import base58
import json
import os
from getpass import getpass

print("🔐 Configurare Securizată\n")

# Colectăm cheile în mod securizat
moralis_key = getpass("📊 Introdu Moralis API Key: ")
helius_key = getpass("🌐 Introdu Helius API Key (sau Enter pentru skip): ") or "demo"

print("\n💰 Configurare Wallet")
print("Exportă cheia privată din Phantom Wallet:")
print("1. Deschide Phantom → Settings → Security & Privacy")
print("2. Show Secret Recovery Phrase sau Export Private Key")
print("3. Copiază cheia (format base58)\n")

phantom_key = getpass("🔑 Introdu Phantom Private Key: ")

# Convertim cheia
try:
    secret_bytes = base58.b58decode(phantom_key)
    secret_list = list(secret_bytes)
    
    # Salvăm în .env
    with open(".env", "w") as f:
        f.write(f'HELIUS_KEY={helius_key}\n')
        f.write(f'MORALIS_KEY={moralis_key}\n')
        f.write(f'SOL_SECRET={json.dumps(secret_list)}\n')
    
    print("\n✅ Configurare salvată cu succes!")
    print("📁 Fișier .env creat")
    
except Exception as e:
    print(f"\n❌ Eroare la configurare: {e}")
    print("Verifică formatul cheii private!")

## 🧪 Pasul 3: Verificare Sistem

In [ ]:
# Verificăm că totul este configurat corect
!python test_bot.py

## 💰 Pasul 4: Verificare Balanță Wallet

In [ ]:
import asyncio
from src.bot.config import settings
from solana.rpc.async_api import AsyncClient

async def check_wallet_balance():
    if not settings.keypair:
        print("❌ Wallet neconfigurat!")
        return
        
    print(f"👛 Wallet Address: {settings.public_key}")
    print("\n💰 Verificare balanță...\n")
    
    client = AsyncClient("https://api.mainnet-beta.solana.com")
    
    try:
        # SOL Balance
        balance = await client.get_balance(settings.keypair.pubkey())
        sol_balance = balance.value / 1e9
        
        print(f"SOL: {sol_balance:.4f} SOL")
        
        if sol_balance < 0.05:
            print("\n⚠️ ATENȚIE: Ai nevoie de minim 0.05 SOL pentru fees!")
            print("Transferă SOL în wallet înainte de a începe trading.")
        else:
            print("\n✅ Balanță SOL suficientă pentru fees")
            
        # Note about USDC
        print("\n📝 Pentru trading ai nevoie și de USDC în wallet.")
        print("Recomandăm minim 10-50 USDC pentru teste.")
        
    finally:
        await client.close()

# Rulăm verificarea
await check_wallet_balance()

## 🚀 Pasul 5: Pornire Bot

In [ ]:
import subprocess
import time
import nest_asyncio

# Permitem async în Jupyter
nest_asyncio.apply()

print("🚀 Pornire servicii...\n")

# Pornim API server în background
api_process = subprocess.Popen(
    ["python", "-m", "src.api.main"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

print("✅ API Server pornit")
time.sleep(3)

# Verificăm că API-ul răspunde
!curl -s http://localhost:8000/ || echo "Waiting for API..."

print("\n📊 Pornire Dashboard...")
print("\n" + "="*50)
print("🌐 Dashboard-ul va fi disponibil la link-ul de mai jos:")
print("="*50 + "\n")

# Pornim dashboard cu share link
!python gradio_ui.py --share

## 📊 Monitorizare Live (Opțional)\n\nRulează celula de mai jos pentru a vedea date în timp real direct în notebook:

In [ ]:
import asyncio
from IPython.display import clear_output
from src.bot import FeedAggregator, TradingEngine
import time

async def live_monitor(duration_seconds=60):
    feeds = FeedAggregator()
    engine = TradingEngine(feeds)
    
    await feeds.start()
    await engine.start()
    
    start_time = time.time()
    
    try:
        while (time.time() - start_time) < duration_seconds:
            clear_output(wait=True)
            
            print("🚀 SOLANA SNIPER BOT - LIVE MONITOR")
            print("=" * 50)
            
            # Token-uri top
            print("\n🎯 Top Oportunități:\n")
            tokens = feeds.get_top_tokens(5)
            
            if tokens:
                for i, token in enumerate(tokens, 1):
                    print(f"{i}. {token.symbol} ({token.address[:8]}...)")
                    print(f"   Preț: ${token.price:.6f} | Schimbare 5m: {token.price_change_5m:+.2f}%")
                    print(f"   Volum 5m: ${token.volume_5m:,.0f} | Score: {token.score:.0f}/100")
                    print()
            else:
                print("Căutare token-uri...")
            
            # Statistici
            stats = engine.get_stats()
            print("\n📈 Statistici Trading:")
            print(f"Poziții active: {stats['positions']}")
            print(f"Total investit: ${stats['total_invested']:.2f}")
            print(f"PnL Total: ${stats['total_pnl']:+.2f} ({stats['roi_percent']:+.1f}%)")
            
            elapsed = int(time.time() - start_time)
            remaining = duration_seconds - elapsed
            print(f"\n⏱️ Timp rămas: {remaining}s")
            
            await asyncio.sleep(5)
            
    finally:
        await engine.stop()
        await feeds.stop()
        print("\n✅ Monitorizare oprită")

# Monitorizează pentru 60 secunde
print("📊 Pornire monitorizare live pentru 60 secunde...")
await live_monitor(60)

## 🛠️ Configurare Avansată

In [ ]:
# Modifică parametrii de trading
config = {
    "position_size": 10.0,    # USDC per poziție
    "max_positions": 5,       # Poziții simultane maxime
    "take_profit": 50.0,      # Target profit %
    "stop_loss": -20.0,       # Stop loss %
}

print("⚙️ Configurare curentă:")
for key, value in config.items():
    print(f"{key}: {value}")

print("\n📝 Pentru a modifica, editează src/bot/trading.py")

## 🛑 Oprire Bot

In [ ]:
# Oprește serviciile când termini
try:
    api_process.terminate()
    print("✅ Servicii oprite")
except:
    print("Serviciile sunt deja oprite")

## 📚 Resurse și Suport\n\n- **GitHub**: [github.com/YOUR_USERNAME/solana-sniper-bot](https://github.com/YOUR_USERNAME/solana-sniper-bot)\n- **Issues**: Raportează probleme pe GitHub\n- **Trading Safety**: Începe cu sume mici și învață gradual\n\n## ⚠️ Reminder Final\n\n- Trading-ul de meme-coins este **EXTREM DE RISCANT**\n- Nu investi mai mult decât îți permiți să pierzi\n- Acest bot este pentru **scop educațional**\n- Monitorizează întotdeauna pozițiile active\n\n**Happy Trading & Stay Safe! 🚀**